In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sys
import sklearn
from sklearn import metrics
import os
import pandas as pd
import time
from sklearn.manifold import TSNE
from collections import  Counter
import math
import logging
logging.disable(30)
from test_step   import test
from train_step4 import train4
from MINE import Estimator

In [2]:
# one client
def weight_add1(model1,test_model):
    for i in range(len(model1.weights)):
        tf.compat.v1.assign(test_model.weights[i],model1.weights[i])
    return model1,test_model
# two clients
def weight_add2(model1,model2,test_model,w1,w2):
    for i in range(len(model1.weights)):
        new_weights=tf.reduce_sum([model1.weights[i]*w1,model2.weights[i]*w2],axis=0)
        tf.compat.v1.assign(model1.weights[i],new_weights)
        tf.compat.v1.assign(model2.weights[i],new_weights)
        tf.compat.v1.assign(test_model.weights[i],new_weights)
    return model1,model2,test_model
# three clients
def weight_add3(model1,model2,model3,test_model,w1,w2,w3):
    for i in range(len(model1.weights)):
        new_weights=tf.reduce_sum([model1.weights[i]*w1,model2.weights[i]*w2,model3.weights[i]*w3],axis=0)
        tf.compat.v1.assign(model1.weights[i],new_weights)
        tf.compat.v1.assign(model2.weights[i],new_weights)
        tf.compat.v1.assign(model3.weights[i],new_weights)
        tf.compat.v1.assign(test_model.weights[i],new_weights)
    return model1,model2,model3,test_model

In [3]:
def retrieve_name(var):
    import inspect
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    var_name_list = [var_name for var_name, var_val in callers_local_vars if var_val is var] 
    if len(var_name_list) > 0:
        return var_name_list[0]

In [4]:
def normalized_preprocess (data):
    data=(data-data.min(axis=1).reshape((len(data),1)))/(data.max(axis=1).reshape((len(data),1))-data.min(axis=1).reshape((len(data),1)))
    return data

In [5]:
#Gear_dataset
G_data1=pd.read_csv(r"F:\数据集样本划分\齿轮数据\DDS试验台--测点 工况\变负载齿轮数据集-3072\data_load0.csv").values
G_label1=pd.read_csv(r"F:\数据集样本划分\齿轮数据\DDS试验台--测点 工况\变负载齿轮数据集-3072\label_load0.csv").values
G_data2=pd.read_csv(r"F:\数据集样本划分\齿轮数据\DDS试验台--测点 工况\变负载齿轮数据集-3072\data_load1.csv").values
G_label2=pd.read_csv(r"F:\数据集样本划分\齿轮数据\DDS试验台--测点 工况\变负载齿轮数据集-3072\label_load1.csv").values
G_data3=pd.read_csv(r"F:\数据集样本划分\齿轮数据\DDS试验台--测点 工况\变负载齿轮数据集-3072\data_load2.csv").values
G_label3=pd.read_csv(r"F:\数据集样本划分\齿轮数据\DDS试验台--测点 工况\变负载齿轮数据集-3072\label_load2.csv").values
G_data4=pd.read_csv(r"F:\数据集样本划分\齿轮数据\DDS试验台--测点 工况\变负载齿轮数据集-3072\data_load3.csv").values
G_label4=pd.read_csv(r"F:\数据集样本划分\齿轮数据\DDS试验台--测点 工况\变负载齿轮数据集-3072\label_load3.csv").values
G_data1=normalized_preprocess(G_data1) 
G_data2=normalized_preprocess(G_data2)
G_data3=normalized_preprocess(G_data3)
G_data4=normalized_preprocess(G_data4)
G_data1=tf.expand_dims(G_data1,axis=-1)
G_data2=tf.expand_dims(G_data2,axis=-1)
G_data3=tf.expand_dims(G_data3,axis=-1)
G_data4=tf.expand_dims(G_data4,axis=-1)
G_train_dataset=tf.data.Dataset.from_tensor_slices((G_data2,G_data1,G_data3,G_data4,G_label2,G_label1,G_label3,G_label4))
G_train_dataset=G_train_dataset.shuffle(5000).batch(500,drop_remainder=True)
print(G_train_dataset)


#Bearing_dataset
B_data1=pd.read_csv(r'F:\数据集样本划分\轴承数据\cwru\12k_de_cwru_data_load0_0.007_test.csv').values
B_label1=pd.read_csv(r'F:\数据集样本划分\轴承数据\cwru\12k_de_cwru_label_load0_0.007_test.csv').values
B_data2=pd.read_csv(r'F:\数据集样本划分\轴承数据\cwru\12k_de_cwru_data_load1_0.007_test.csv').values
B_label2=pd.read_csv(r'F:\数据集样本划分\轴承数据\cwru\12k_de_cwru_label_load1_0.007_test.csv').values
B_data3=pd.read_csv(r'F:\数据集样本划分\轴承数据\cwru\12k_de_cwru_data_load2_0.007_test.csv').values
B_label3=pd.read_csv(r'F:\数据集样本划分\轴承数据\cwru\12k_de_cwru_label_load2_0.007_test.csv').values
B_data4=pd.read_csv(r'F:\数据集样本划分\轴承数据\cwru\12k_de_cwru_data_load3_0.007_test.csv').values
B_label4=pd.read_csv(r'F:\数据集样本划分\轴承数据\cwru\12k_de_cwru_label_load3_0.007_test.csv').values
B_data1=normalized_preprocess(B_data1) 
B_data2=normalized_preprocess(B_data2)
B_data3=normalized_preprocess(B_data3)
B_data4=normalized_preprocess(B_data4)
B_data1=tf.expand_dims(B_data1,axis=-1)
B_data2=tf.expand_dims(B_data2,axis=-1)
B_data3=tf.expand_dims(B_data3,axis=-1)
B_data4=tf.expand_dims(B_data4,axis=-1)
B_train_dataset=tf.data.Dataset.from_tensor_slices((B_data1,B_data2,B_data3,B_data4,B_label1,B_label2,B_label3,B_label4))
B1_train_dataset=B_train_dataset.shuffle(5000).batch(500,drop_remainder=True)
print(B1_train_dataset)


#Bearing_dataset
B_data1=pd.read_csv(r"F:\数据集样本划分\轴承数据\cqu\data_load0_2000_channel1_test.csv").values
B_label1=pd.read_csv(r"F:\数据集样本划分\轴承数据\cqu\label_load0_2000_channel1_test.csv").values
B_data2=pd.read_csv(r"F:\数据集样本划分\轴承数据\cqu\data_load1_2000_channel1_test.csv").values
B_label2=pd.read_csv(r"F:\数据集样本划分\轴承数据\cqu\label_load1_2000_channel1_test.csv").values
B_data3=pd.read_csv(r"F:\数据集样本划分\轴承数据\cqu\data_load2_2000_channel1_test.csv").values
B_label3=pd.read_csv(r"F:\数据集样本划分\轴承数据\cqu\label_load2_2000_channel1_test.csv").values
B_data4=pd.read_csv(r"F:\数据集样本划分\轴承数据\cqu\data_load3_1000_channel1_test.csv").values
B_label4=pd.read_csv(r"F:\数据集样本划分\轴承数据\cqu\label_load3_1000_channel1_test.csv").values
B_data1=normalized_preprocess(B_data1) 
B_data2=normalized_preprocess(B_data2)
B_data3=normalized_preprocess(B_data3)
B_data4=normalized_preprocess(B_data4)
B_data1=tf.expand_dims(B_data1,axis=-1)
B_data2=tf.expand_dims(B_data2,axis=-1)
B_data3=tf.expand_dims(B_data3,axis=-1)
B_data4=tf.expand_dims(B_data4,axis=-1)
B_train_dataset=tf.data.Dataset.from_tensor_slices((B_data1,B_data2,B_data3,B_data4,B_label1,B_label2,B_label3,B_label4))
B2_train_dataset=B_train_dataset.shuffle(5000).batch(500,drop_remainder=True)
print(B2_train_dataset)


#Bearing_dataset
B_data1=pd.read_csv(r"F:\数据集样本划分\轴承数据\swju\data_load0_1000_0.3_test.csv").values
B_label1=pd.read_csv(r"F:\数据集样本划分\轴承数据\swju\label_load0_1000_0.3_test.csv").values
B_data2=pd.read_csv(r"F:\数据集样本划分\轴承数据\swju\data_load1_1000_0.3_test.csv").values
B_label2=pd.read_csv(r"F:\数据集样本划分\轴承数据\swju\label_load1_1000_0.3_test.csv").values
B_data3=pd.read_csv(r"F:\数据集样本划分\轴承数据\swju\data_load2_1000_0.3_test.csv").values
B_label3=pd.read_csv(r"F:\数据集样本划分\轴承数据\swju\label_load2_1000_0.3_test.csv").values
B_data4=pd.read_csv(r"F:\数据集样本划分\轴承数据\swju\data_load3_1000_0.3_test.csv").values
B_label4=pd.read_csv(r"F:\数据集样本划分\轴承数据\swju\label_load3_1000_0.3_test.csv").values
B_data1=normalized_preprocess(B_data1) 
B_data2=normalized_preprocess(B_data2)
B_data3=normalized_preprocess(B_data3)
B_data4=normalized_preprocess(B_data4)
print(B_data1[0][0],B_data2[0][0],B_data3[0][0],B_data4[0][0])
B_data1=tf.expand_dims(B_data1,axis=-1)
B_data2=tf.expand_dims(B_data2,axis=-1)
B_data3=tf.expand_dims(B_data3,axis=-1)
B_data4=tf.expand_dims(B_data4,axis=-1)
B_train_dataset=tf.data.Dataset.from_tensor_slices((B_data3,B_data1,B_data2,B_data4,B_label3,B_label1,B_label2,B_label4))
B3_train_dataset=B_train_dataset.shuffle(5000).batch(500,drop_remainder=True)
print(B3_train_dataset)

<BatchDataset shapes: ((500, 3072, 1), (500, 3072, 1), (500, 3072, 1), (500, 3072, 1), (500, 5), (500, 5), (500, 5), (500, 5)), types: (tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64)>
<BatchDataset shapes: ((500, 3072, 1), (500, 3072, 1), (500, 3072, 1), (500, 3072, 1), (500, 4), (500, 4), (500, 4), (500, 4)), types: (tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64)>
<BatchDataset shapes: ((500, 3072, 1), (500, 3072, 1), (500, 3072, 1), (500, 3072, 1), (500, 4), (500, 4), (500, 4), (500, 4)), types: (tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64)>
0.5341327554135927 0.5920691489895694 0.5684197615146983 0.7179127363638874
<BatchDataset shapes: ((500, 3072, 1), (500, 3072, 1), (500, 3072, 1), (500, 3072, 1), (500, 4), (500, 4), (500, 4), (500, 4)), types: (tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.fl

In [6]:
def Feature_extractor():
    model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32,kernel_size=16,strides=8,activation='relu',padding='same',input_shape=(3072,1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same'),

        tf.keras.layers.Conv1D(filters=64,kernel_size=3,strides=1,activation='relu',padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same'),

        tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=1,activation='relu',padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same'),])
    return model

def Disentangler():
    input_data=tf.keras.Input(shape=(48,128))
    x1=tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=1,activation='relu',padding='same')(input_data)
    x1=tf.keras.layers.BatchNormalization()(x1)
    x1=tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same')(x1)
    x1=tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=1,activation='relu',padding='same')(x1)
    x1=tf.keras.layers.BatchNormalization()(x1)
    x1=tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same')(x1)    
    x1=tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=1,activation='relu',padding='same')(x1)
    x1=tf.keras.layers.BatchNormalization()(x1)
    ci_output=tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same')(x1)
    
    x2=tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=1,activation='relu',padding='same')(input_data)
    x2=tf.keras.layers.BatchNormalization()(x2)
    x2=tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same')(x2)
    x2=tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=1,activation='relu',padding='same')(x2)
    x2=tf.keras.layers.BatchNormalization()(x2)
    x2=tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same')(x2)    
    x2=tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=1,activation='relu',padding='same')(x2)
    x2=tf.keras.layers.BatchNormalization()(x2)
    cr_output=tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same')(x2)
    
    model=tf.keras.Model(inputs=input_data, outputs=[ci_output,cr_output])
    return model

def Reconstructor():
    ci_input=tf.keras.Input(shape=(6,128))
    x1=tf.keras.layers.Conv1DTranspose(filters=128,kernel_size=3,strides=2,activation='relu',padding='same')(ci_input)
    x1=tf.keras.layers.BatchNormalization()(x1)
    x1=tf.keras.layers.Conv1DTranspose(filters=128,kernel_size=3,strides=2,activation='relu',padding='same')(x1)
    x1=tf.keras.layers.BatchNormalization()(x1)
    x1=tf.keras.layers.Conv1DTranspose(filters=128,kernel_size=3,strides=2,activation='relu',padding='same')(x1)
    ci_recon=tf.keras.layers.BatchNormalization()(x1)
    
    cr_input=tf.keras.Input(shape=(6,128))
    x2=tf.keras.layers.Conv1DTranspose(filters=128,kernel_size=3,strides=2,activation='relu',padding='same')(cr_input)
    x2=tf.keras.layers.BatchNormalization()(x2)
    x2=tf.keras.layers.Conv1DTranspose(filters=128,kernel_size=3,strides=2,activation='relu',padding='same')(x2)
    x2=tf.keras.layers.BatchNormalization()(x2)
    x2=tf.keras.layers.Conv1DTranspose(filters=128,kernel_size=3,strides=2,activation='relu',padding='same')(x2)
    cr_recon=tf.keras.layers.BatchNormalization()(x2)
    
    recon_data=tf.keras.layers.Add()([ci_recon,cr_recon])
    model=tf.keras.Model(inputs=[ci_input,cr_input], outputs=recon_data)
    return model


def Classifier(unit_number):
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(128,activation='relu',input_shape=(128,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(unit_number,activation='relu'),])
    return model


def NET():
    model=tf.keras.Sequential([
          tf.keras.layers.Dense(8,input_shape=(256,),activation='relu'),
          tf.keras.layers.Dense(4,activation='relu'),
          tf.keras.layers.Dense(1),])
    return model

In [7]:
def comunication(comunication_number):
    GAP=tf.keras.layers.GlobalAveragePooling1D()
    G_feature_extractor=Feature_extractor()
    G_disentangler=Disentangler()
    G_reconstructor=Reconstructor()
    G_classifier=Classifier(5)
    G_net=NET()

    B1_feature_extractor=Feature_extractor()
    B1_disentangler=Disentangler()
    B1_reconstructor=Reconstructor()
    B1_classifier=Classifier(4)
    B1_net=NET()

    B2_feature_extractor=Feature_extractor()
    B2_disentangler=Disentangler()
    B2_reconstructor=Reconstructor()
    B2_classifier=Classifier(4)
    B2_net=NET() 
    
    B3_feature_extractor=Feature_extractor()
    B3_disentangler=Disentangler()
    B3_reconstructor=Reconstructor()
    B3_classifier=Classifier(4)
    B3_net=NET()
    
    weights={"w1":[],"w2":[],"w3":[],"test2":[],"test3":[],"test4":[]}
    for i in range(comunication_number):
        print("*"*100)
        loss1=train4(B1_feature_extractor,B1_disentangler,B1_reconstructor,B1_classifier,B1_net,5,1,1,1,B1_train_dataset,100)
        loss2=train4(B2_feature_extractor,B2_disentangler,B2_reconstructor,B2_classifier,B2_net,5,1,1,1,B2_train_dataset,100)
        loss3=train4(G_feature_extractor,G_disentangler,G_reconstructor,G_classifier,G_net,5,1,1,1,G_train_dataset,100)
        test_loss,orig_feature,label_feature=test(B3_feature_extractor,B3_disentangler,B3_reconstructor,B3_classifier,B3_net,5,1,1,1,B3_train_dataset,100)        
        
        ci1,cr1= B1_disentangler(orig_feature,training = True)
        label_feature1=B1_classifier(GAP(cr1),training = True)
        info1 = Estimator(label_feature1.shape[-1],label_feature.shape[-1]).backward(label_feature1,label_feature,2000)
        
        ci1,cr1= B2_disentangler(orig_feature,training = True)
        label_feature2=B2_classifier(GAP(cr1),training = True)
        info2 = Estimator(label_feature2.shape[-1],label_feature.shape[-1]).backward(label_feature2,label_feature,2000)
 
        ci1,cr1= G_disentangler(orig_feature,training = True)
        label_feature3=G_classifier(GAP(cr1),training = True)
        info3 = Estimator(label_feature3.shape[-1],label_feature.shape[-1]).backward(label_feature3,label_feature,2000)
        
        c=0.2
        w1=tf.nn.softmax([info1[-1]/c,info2[-1]/c,info3[-1]/c],axis=0)[0]
        w2=tf.nn.softmax([info1[-1]/c,info2[-1]/c,info3[-1]/c],axis=0)[1]
        w3=tf.nn.softmax([info1[-1]/c,info2[-1]/c,info3[-1]/c],axis=0)[2]
        
        B1_feature_extractor,B2_feature_extractor,G_feature_extractor,B3_feature_extractor=weight_add3(B1_feature_extractor,B2_feature_extractor,
                                                                                                       G_feature_extractor,B3_feature_extractor,w1,w2,w3)
        
        weights["w1"].append(np.array(w1))
        weights["w2"].append(np.array(w2))
        weights["w3"].append(np.array(w3))
        weights["test2"].append(np.array(test_loss["cr_acc2"][-1]))
        weights["test3"].append(np.array(test_loss["cr_acc3"][-1]))
        weights["test4"].append(np.array(test_loss["cr_acc4"][-1]))
        print(info1[-1].numpy(),info2[-1].numpy(),info3[-1].numpy(),w1.numpy(),w2.numpy(),w3.numpy(),test_loss["cr_acc2"][-1],test_loss["cr_acc3"][-1],test_loss["cr_acc4"][-1])
    return loss1,loss2,loss3,test_loss,weights

In [8]:
loss1,loss2,loss3,test_loss,weights=comunication(10)

****************************************************************************************************
1.06051 0.4393561 0.54069185 0.8935503 0.04002214 0.06642754 0.871 0.868 0.94025
****************************************************************************************************
1.130262 1.0143768 1.1692249 0.36032078 0.20185879 0.43782043 0.906 0.92025 0.9705
****************************************************************************************************
0.95863885 1.3413398 1.2087466 0.08873886 0.60136527 0.30989584 0.907 0.9055 0.9485
****************************************************************************************************
0.9941937 1.7689666 0.66852146 0.020274002 0.9757472 0.0039787935 0.9505 0.95075 0.99175
****************************************************************************************************
1.1712208 1.8101406 1.041736 0.038574617 0.9412357 0.020189662 0.93325 0.92225 0.98875
***********************************************************************